In [1]:
import math
import re
import sys
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, auc, matthews_corrcoef, precision_score, recall_score, confusion_matrix, roc_curve,roc_auc_score
from torch.nn import GRU, Linear, ReLU
%config Warning.simplerfilter = True

# device =torch.device("cuda:0"if torch.cuda.is_available() else "cpu")
# device = torch.device('cuda:0')
device = torch.device('cpu')

## 初始数据集的特征

In [93]:
ori_fea = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\初始BiGRU所有特征.csv')
# ori_fea = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\初始transformer所有特征-10.csv')
# ori_fea = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\初始transformer所有特征-45.csv')
ori_fea = ori_fea.drop(ori_fea.columns[0], axis=1)

label = [1] * 8000 + [0] * 8000

oy = pd.DataFrame(label, columns=['label'])
ori_fea

,GC_con,score,cdsStop,cdsSizes,T,C,GT,GC,ACG,TAT,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,0.380952,272.0,182,72,0.273016,0.155556,0.051118,0.041534,0.003215,0.006431,...,0.217313,0.415759,0.207001,0.484941,0.213454,0.470569,0.178506,0.459906,0.208311,0.480269
1,0.409396,454.0,432,354,0.328859,0.177852,0.079125,0.035354,0.011824,0.016892,...,0.253189,0.613251,0.253189,0.613251,0.253189,0.613251,0.253189,0.613251,0.253189,0.613251
2,0.343101,506.0,585,339,0.344322,0.191697,0.046512,0.022032,0.008589,0.031902,...,0.484185,0.416200,0.352587,0.524192,0.362477,0.509556,0.293591,0.458166,0.354463,0.523748
3,0.461774,427.0,327,327,0.293578,0.299694,0.036923,0.046154,0.000000,0.000000,...,0.425386,0.601465,0.425386,0.601465,0.425386,0.601465,0.425386,0.601465,0.425386,0.601465
4,0.565217,0.0,0,0,0.304348,0.347826,0.095238,0.047619,0.000000,0.000000,...,0.575878,0.469469,0.516594,0.525154,0.494258,0.510499,0.303489,0.491147,0.474840,0.531109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0.450256,1175.0,975,975,0.274872,0.217436,0.047276,0.057554,0.007209,0.011329,...,0.701008,0.536703,0.701008,0.536703,0.701008,0.536703,0.701008,0.536703,0.701008,0.536703
15996,0.430957,1757.0,1557,1557,0.280026,0.185613,0.057235,0.051447,0.004507,0.011590,...,0.687417,0.500093,0.687417,0.500093,0.687417,0.500093,0.687417,0.500093,0.687417,0.500093
15997,0.455010,3134.0,2934,2934,0.233810,0.190184,0.042974,0.057981,0.005119,0.012287,...,0.687417,0.500093,0.687417,0.500093,0.687417,0.500093,0.687417,0.500093,0.687417,0.500093
15998,0.448404,2894.0,2694,2694,0.257981,0.213066,0.044948,0.048663,0.002230,0.011524,...,0.667058,0.442191,0.667058,0.442191,0.667058,0.442191,0.667058,0.442191,0.667058,0.442191


In [198]:
def extract_sequences_from_fasta(file_path):
    headers = []
    sequences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        current_header = ''
        current_sequence = ''
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if current_header and current_sequence:
                    headers.append(current_header)
                    sequences.append(current_sequence)

                current_header = line[1:]
                current_sequence = ''
            else:
                current_sequence += line

        # 处理最后一个序列
        if current_header and current_sequence:
            headers.append(current_header)
            sequences.append(current_sequence)

    data = {'seqname': headers, 'seq': sequences}
    df = pd.DataFrame(data)
    return df

# 读取FASTA文件并提取序列
file_path = r'C:\Users\26970\Desktop\package\DL\PINC-main\DataSets\TestSets\\cRNA\Solanum tuberosum.fasta'
datah = extract_sequences_from_fasta(file_path)
print(datah.shape[0])

file_path = r'C:\Users\26970\Desktop\package\DL\PINC-main\DataSets\TestSets\\ncRNA\Solanum tuberosum.fasta'
datah_nc = extract_sequences_from_fasta(file_path)
print(datah_nc.shape[0])
datah = pd.concat([datah,datah_nc]).reset_index(drop=True)

pattern = r'(?:^|\s)([^ ]+)'  # 匹配第一个空格前的非空格字符序列作为标识部分
datah['seqname'] = [re.search(pattern, identifier).group(1) for identifier in datah['seqname']]

data = datah
datah

8282
8282


,seqname,seq
0,PGSC0003DMT400040871,ATGGCTATGGCATCGACGGAGAGAGTGATTCTTGGTTCAATTGCTT...
1,PGSC0003DMT400008138,ATGGACAAAGAACAAAAGACCGAGCACAATGATGAATTTGTGATGG...
2,PGSC0003DMT400081965,ATGCCGTTTATAACACTTTGGAGTTTAAGATGCGGTATCAAAACAA...
3,PGSC0003DMT400065837,ATGGGAGTGAAGTTAGTTTTTTCGATCCTCATAGTGACTATGATGA...
4,PGSC0003DMT400075266,ATGGCTTTACAACAACCATTTCTATCTGAATTACTCCACTCATCTT...
...,...,...
16559,URS0001BC9DCE_4113,UUACUGGUGGGUGGCGGAGAAGAGAUCGUGACAUCGUAGACCUAUG...
16560,URS0001BCA1A5_4113,GGGCUUAUAGUUUAAUUGGUUGAAACGUACCGCUCAUAACGGUGAU...
16561,URS00021C2229_4113,CTATGATGGATGAAGGCCTTTCATTTTGGCTCTGTAGGGCTATCCA...
16562,URS00021C2229_80418,CTATGATGGATGAAGGCCTTTCATTTTGGCTCTGTAGGGCTATCCA...


In [199]:
names = ['seqname', 'start', 'end', 'source ', 'accession', 'score ', 'startComplete ',
                                 'endComplete ', 'cdsCount ', 'cdsStarts ', 'cdsSizes ']
index_col = "seqname"

rna_cds = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\\ncRNA_test\c_cds_test\Solanum_tuberosum',names=names,sep='\t')
print(rna_cds.shape[0])
rna_cds_nc = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\\ncRNA_test\\nc_cds_test\Solanum_tuberosum',names=names,sep='\t')
print(rna_cds_nc.shape[0])

rna_cds = pd.concat([rna_cds,rna_cds_nc]).reset_index(drop=True)

datah_seqname = datah['seqname']
rna_cds_seqname = rna_cds['seqname']

# # extra_rows_index = datah_seqname.difference(rna_cds_seqname)
indexes_to_drop = datah_seqname[~datah_seqname.isin(rna_cds_seqname)].index

# 删除行
datah.drop(indexes_to_drop, inplace=True)

# 创建一个包含每个打印的元素的列表
label = [0] * 8282 + [1] * 7838

# 使用pandas库将列表转换为DataFrame
y = pd.DataFrame(label, columns=['label'])
datah.reset_index(drop=True, inplace=True)

datah.insert(0,'label',y)
print(datah.index)
print(y.index)

data3 = datah.copy()
datah

8282
7838
RangeIndex(start=0, stop=16120, step=1)
RangeIndex(start=0, stop=16120, step=1)


,label,seqname,seq
0,0,PGSC0003DMT400040871,ATGGCTATGGCATCGACGGAGAGAGTGATTCTTGGTTCAATTGCTT...
1,0,PGSC0003DMT400008138,ATGGACAAAGAACAAAAGACCGAGCACAATGATGAATTTGTGATGG...
2,0,PGSC0003DMT400081965,ATGCCGTTTATAACACTTTGGAGTTTAAGATGCGGTATCAAAACAA...
3,0,PGSC0003DMT400065837,ATGGGAGTGAAGTTAGTTTTTTCGATCCTCATAGTGACTATGATGA...
4,0,PGSC0003DMT400075266,ATGGCTTTACAACAACCATTTCTATCTGAATTACTCCACTCATCTT...
...,...,...,...
16115,1,URS0000E3F3BF_4113,AAGCGGGGTAGAGGAATTGGTCAACTCATCAGGCTCATGACCTGAA...
16116,1,URS0000E44DF0_4113,CTATGATGGATGAAGGCCTTTCATTTTTGCTCTGTAGGGCTATCCA...
16117,1,URS00021C2229_4113,CTATGATGGATGAAGGCCTTTCATTTTGGCTCTGTAGGGCTATCCA...
16118,1,URS00021C2229_80418,CTATGATGGATGAAGGCCTTTCATTTTGGCTCTGTAGGGCTATCCA...


## 手动特征

In [200]:
from itertools import product
# compute k-mers features (k=1~4)
def Kmers_funct(seq, x):
    X = [None] * len(seq)
    for i in range(len(seq)):
        a = seq[i]
        t = 0
        l = []
        for index in range(len(a) - x + 1):
            t = a[index:index + x]
            if (len(t)) == x:
                l.append(t)
        X[i] = l
    return X


def nucleotide_type(k):
    z = []
    for i in product('ACGT', repeat=k):
        z.append(''.join(i))
    return z


def Kmers_frequency(seq, x):
    X = []
    char = nucleotide_type(x)
    for i in range(len(seq)):
        s = seq[i]
        frequence = []
        for a in char:
            number = s.count(a)
            char_frequence = number / (len(s) - x + 1)
            frequence.append(char_frequence)
        X.append(frequence)
    return X


feature_1mer = Kmers_funct(data.seq, 1)  # 1-mer
feature_2mer = Kmers_funct(data.seq, 2)  # 2-mer
feature_3mer = Kmers_funct(data.seq, 3)  # 3-mer

feature_1mer_frequency = Kmers_frequency(feature_1mer, 1)  # 1-mer
feature_2mer_frequency = Kmers_frequency(feature_2mer, 2)  # 2-mer
feature_3mer_frequency = Kmers_frequency(feature_3mer, 3)  # 3-mer

feature = pd.concat([pd.DataFrame(feature_1mer_frequency),
                     pd.DataFrame(feature_2mer_frequency),
                     pd.DataFrame(feature_3mer_frequency)], axis=1)
feature.columns = ['A','C','G','T','AA', 'AC', 'AG', 'AT', 'CA', 'CC', 'CG', 'CT', 'GA', 'GC', 'GG', 'GT', 'TA', 'TC', 'TG', 'TT','AAA', 'AAC', 'AAG', 'AAT', 'ACA', 'ACC', 'ACG', 'ACT', 'AGA', 'AGC', 'AGG', 'AGT', 'ATA', 'ATC', 'ATG', 'ATT', 'CAA', 'CAC', 'CAG', 'CAT', 'CCA', 'CCC', 'CCG', 'CCT', 'CGA', 'CGC', 'CGG', 'CGT', 'CTA', 'CTC', 'CTG', 'CTT', 'GAA', 'GAC', 'GAG', 'GAT', 'GCA', 'GCC', 'GCG', 'GCT', 'GGA', 'GGC', 'GGG', 'GGT', 'GTA', 'GTC', 'GTG', 'GTT', 'TAA', 'TAC', 'TAG', 'TAT', 'TCA', 'TCC', 'TCG', 'TCT', 'TGA', 'TGC', 'TGG', 'TGT', 'TTA', 'TTC', 'TTG', 'TTT']
feature

,A,C,G,T,AA,AC,AG,AT,CA,CC,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.273810,0.151515,0.227273,0.347403,0.085683,0.030369,0.063991,0.094360,0.056399,0.018438,...,0.009783,0.021739,0.023913,0.021739,0.021739,0.031522,0.022826,0.020652,0.040217,0.046739
1,0.326923,0.177350,0.241453,0.254274,0.111588,0.066524,0.079399,0.070815,0.070815,0.036481,...,0.004310,0.010776,0.025862,0.017241,0.034483,0.012931,0.015086,0.019397,0.030172,0.012931
2,0.305556,0.179167,0.241667,0.273611,0.103064,0.045961,0.079387,0.076602,0.062674,0.030641,...,0.004190,0.016760,0.041899,0.032123,0.013966,0.015363,0.011173,0.013966,0.034916,0.020950
3,0.222222,0.302682,0.172414,0.302682,0.055769,0.071154,0.025000,0.069231,0.065385,0.105769,...,0.005792,0.021236,0.017375,0.013514,0.013514,0.013514,0.017375,0.032819,0.021236,0.030888
4,0.300000,0.190909,0.223636,0.285455,0.107403,0.057039,0.062500,0.072816,0.067961,0.038835,...,0.009113,0.017618,0.022479,0.016403,0.022479,0.020656,0.013973,0.022479,0.028554,0.033414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16115,0.250000,0.250000,0.276316,0.223684,0.067568,0.027027,0.081081,0.067568,0.067568,0.081081,...,0.013889,0.000000,0.027778,0.013889,0.013889,0.013889,0.000000,0.013889,0.027778,0.000000
16116,0.195122,0.207317,0.280488,0.317073,0.025000,0.000000,0.075000,0.100000,0.062500,0.050000,...,0.000000,0.012821,0.025641,0.012821,0.038462,0.012821,0.000000,0.025641,0.025641,0.064103
16117,0.195122,0.207317,0.292683,0.304878,0.025000,0.000000,0.075000,0.100000,0.062500,0.050000,...,0.000000,0.012821,0.025641,0.000000,0.051282,0.012821,0.000000,0.025641,0.025641,0.051282
16118,0.195122,0.207317,0.292683,0.304878,0.025000,0.000000,0.075000,0.100000,0.062500,0.050000,...,0.000000,0.012821,0.025641,0.000000,0.051282,0.012821,0.000000,0.025641,0.025641,0.051282


In [201]:
def gc_content(dna_sequence):
    gc_percent = []
    for i in dna_sequence:
        gc_count = i.count('G') + i.count('C')
        total_count = len(i)
        gc_percent_eve = (gc_count / total_count) * 100
        gc_percent.append(gc_percent_eve)
    return gc_percent

In [202]:
# gc_precent = gc_content(data.iloc[:,0])
gc_precent = gc_content(data.seq)
gc_precent_pd = pd.DataFrame(gc_precent)
gc_precent_pd.rename(columns={0: 'GC content'}, inplace=True)
print(gc_precent_pd.shape)
gc_precent_pd

feature.insert(0,'GC content',gc_precent_pd)

feature = pd.DataFrame(feature)
feature.iloc[:,0] = feature.iloc[:,0]/100
feature

(16120, 1)


,GC content,A,C,G,T,AA,AC,AG,AT,CA,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.378788,0.273810,0.151515,0.227273,0.347403,0.085683,0.030369,0.063991,0.094360,0.056399,...,0.009783,0.021739,0.023913,0.021739,0.021739,0.031522,0.022826,0.020652,0.040217,0.046739
1,0.418803,0.326923,0.177350,0.241453,0.254274,0.111588,0.066524,0.079399,0.070815,0.070815,...,0.004310,0.010776,0.025862,0.017241,0.034483,0.012931,0.015086,0.019397,0.030172,0.012931
2,0.420833,0.305556,0.179167,0.241667,0.273611,0.103064,0.045961,0.079387,0.076602,0.062674,...,0.004190,0.016760,0.041899,0.032123,0.013966,0.015363,0.011173,0.013966,0.034916,0.020950
3,0.475096,0.222222,0.302682,0.172414,0.302682,0.055769,0.071154,0.025000,0.069231,0.065385,...,0.005792,0.021236,0.017375,0.013514,0.013514,0.013514,0.017375,0.032819,0.021236,0.030888
4,0.414545,0.300000,0.190909,0.223636,0.285455,0.107403,0.057039,0.062500,0.072816,0.067961,...,0.009113,0.017618,0.022479,0.016403,0.022479,0.020656,0.013973,0.022479,0.028554,0.033414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16115,0.526316,0.250000,0.250000,0.276316,0.223684,0.067568,0.027027,0.081081,0.067568,0.067568,...,0.013889,0.000000,0.027778,0.013889,0.013889,0.013889,0.000000,0.013889,0.027778,0.000000
16116,0.487805,0.195122,0.207317,0.280488,0.317073,0.025000,0.000000,0.075000,0.100000,0.062500,...,0.000000,0.012821,0.025641,0.012821,0.038462,0.012821,0.000000,0.025641,0.025641,0.064103
16117,0.500000,0.195122,0.207317,0.292683,0.304878,0.025000,0.000000,0.075000,0.100000,0.062500,...,0.000000,0.012821,0.025641,0.000000,0.051282,0.012821,0.000000,0.025641,0.025641,0.051282
16118,0.500000,0.195122,0.207317,0.292683,0.304878,0.025000,0.000000,0.075000,0.100000,0.062500,...,0.000000,0.012821,0.025641,0.000000,0.051282,0.012821,0.000000,0.025641,0.025641,0.051282


In [203]:
sequence_len = []
for i in range(data.shape[0]):
    sequence_len.append(len(data.seq[i]))
sequence_len = pd.DataFrame(sequence_len)
sequence_len.rename(columns={0:'seq_len'},inplace=True)
sequence_len

feature.insert(1,'seq_len',sequence_len)
feature.rename(columns={'GC content':'GC_con'},inplace=True)
print(feature.shape)
feature

(16120, 86)


,GC_con,seq_len,A,C,G,T,AA,AC,AG,AT,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.378788,924,0.273810,0.151515,0.227273,0.347403,0.085683,0.030369,0.063991,0.094360,...,0.009783,0.021739,0.023913,0.021739,0.021739,0.031522,0.022826,0.020652,0.040217,0.046739
1,0.418803,468,0.326923,0.177350,0.241453,0.254274,0.111588,0.066524,0.079399,0.070815,...,0.004310,0.010776,0.025862,0.017241,0.034483,0.012931,0.015086,0.019397,0.030172,0.012931
2,0.420833,720,0.305556,0.179167,0.241667,0.273611,0.103064,0.045961,0.079387,0.076602,...,0.004190,0.016760,0.041899,0.032123,0.013966,0.015363,0.011173,0.013966,0.034916,0.020950
3,0.475096,522,0.222222,0.302682,0.172414,0.302682,0.055769,0.071154,0.025000,0.069231,...,0.005792,0.021236,0.017375,0.013514,0.013514,0.013514,0.017375,0.032819,0.021236,0.030888
4,0.414545,1650,0.300000,0.190909,0.223636,0.285455,0.107403,0.057039,0.062500,0.072816,...,0.009113,0.017618,0.022479,0.016403,0.022479,0.020656,0.013973,0.022479,0.028554,0.033414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16115,0.526316,76,0.250000,0.250000,0.276316,0.223684,0.067568,0.027027,0.081081,0.067568,...,0.013889,0.000000,0.027778,0.013889,0.013889,0.013889,0.000000,0.013889,0.027778,0.000000
16116,0.487805,82,0.195122,0.207317,0.280488,0.317073,0.025000,0.000000,0.075000,0.100000,...,0.000000,0.012821,0.025641,0.012821,0.038462,0.012821,0.000000,0.025641,0.025641,0.064103
16117,0.500000,82,0.195122,0.207317,0.292683,0.304878,0.025000,0.000000,0.075000,0.100000,...,0.000000,0.012821,0.025641,0.000000,0.051282,0.012821,0.000000,0.025641,0.025641,0.051282
16118,0.500000,82,0.195122,0.207317,0.292683,0.304878,0.025000,0.000000,0.075000,0.100000,...,0.000000,0.012821,0.025641,0.000000,0.051282,0.012821,0.000000,0.025641,0.025641,0.051282


## cds特征

In [204]:
names = ['seqname', 'start', 'end', 'source ', 'accession', 'score ', 'startComplete ',
                                 'endComplete ', 'cdsCount ', 'cdsStarts ', 'cdsSizes ']
index_col = "seqname"

rna_cds = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\\ncRNA_test\c_cds_test\Solanum_tuberosum',names=names,sep='\t', index_col=index_col)
rna_cds = rna_cds
rna_cds_nc = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\\ncRNA_test\\nc_cds_test\Solanum_tuberosum',names=names,sep='\t', index_col=index_col)
rna_cds_nc = rna_cds_nc

rna_cds = pd.concat([rna_cds,rna_cds_nc]).reset_index(drop=True)
rna_cds

data1 = data
data2 = data
for i, l in data2.iterrows():
    data1.iloc[i,2] = len(l['seq'])
    
mylist = []
mylist = pd.DataFrame(mylist,columns=['score','cdsStarts','cdsStop','cdsSizes','cdsPercent'])

for index, row in rna_cds.iterrows():
    tem_pd_score = pd.DataFrame(row)
    mylist.loc[index, 'cdsStarts'] = row["start"]
    mylist.loc[index, 'cdsStop'] = row["end"]
    mylist.loc[index, 'cdsSizes'] = row["end"] - row["start"]
    mylist.loc[index, 'score'] = tem_pd_score.iloc[4, 0]
    mylist.loc[index, 'cdsPercent'] = (row["start"] + row["end"]) / data1.loc[index, 'seq']
    
feature = feature.join(mylist)

# 选取其中10个特征
# feature = feature[['GC_con', 'score', 'cdsStop', 'cdsSizes','T','C','GT','GC','ACG','TAT']]
# 选取其中45个特征
# feature = feature[['GC_con', 'seq_len', 'A', 'C', 'G', 'T', 'AA', 'AC', 'AG', 'AT', 'CA', 'CC', 'CG', 'CT', 'GA', 'GC', 'GG', 'GT', 'TA', 'TC', 'TG', 'TT', 'AAA', 'AAG', 'AGA', 'ATT', 'CCG', 'CGC', 'CGG', 'CTC', 'GAA', 'GAG', 'GCC', 'GCG', 'GGC', 'GGG', 'TCT', 'TGG', 'TTG', 'TTT', 'score', 'cdsStarts', 'cdsStop', 'cdsSizes', 'cdsPercent']]

feature

,GC_con,score,cdsStop,cdsSizes,T,C,GT,GC,ACG,TAT
0,0.378788,1124.0,924,924,0.347403,0.151515,0.063991,0.040130,0.004348,0.023913
1,0.418803,668.0,468,468,0.254274,0.177350,0.057940,0.032189,0.012931,0.010776
2,0.420833,820.0,720,720,0.273611,0.179167,0.051532,0.057103,0.005587,0.015363
3,0.475096,722.0,522,522,0.302682,0.302682,0.044231,0.044231,0.005792,0.017375
4,0.414545,1850.0,1650,1650,0.285455,0.190909,0.049757,0.037015,0.006075,0.015796
...,...,...,...,...,...,...,...,...,...,...
16115,0.526316,109.0,45,9,0.223684,0.250000,0.054054,0.054054,0.000000,0.000000
16116,0.487805,223.5,80,75,0.317073,0.207317,0.037500,0.075000,0.000000,0.025641
16117,0.500000,223.5,80,75,0.304878,0.207317,0.037500,0.075000,0.000000,0.025641
16118,0.500000,223.5,80,75,0.304878,0.207317,0.037500,0.075000,0.000000,0.025641


## 深度特征

In [2]:
data = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\基准数据集.csv')
sequences = data['Sequence']
labels = data.Label.values
# sequences = data3['seq']
# labels = data3['label'].values
print(labels)

pat = re.compile('[AGCTagct]')

def pre_process(text):
    text = pat.findall(text)
    text = [each.lower() for each in text]
    return text

x = sequences.apply(pre_process)
word_set = set()

for lst in x:
    for word in lst:
        word_set.add(word)

word_list = list(word_set)
word_index = dict([(each, word_list.index(each) + 1) for each in word_list])

text = x.apply(lambda x: [word_index.get(word, 0) for word in x])
print(text)

text_len = 1200

pad_text = [l + (text_len - len(l)) * [0] if len(l) < text_len else l[:text_len] for l in text]
pad_text = np.array(pad_text)
pad_text

pad_text,labels=torch.LongTensor(pad_text),torch.LongTensor(labels)
x_train, x_test, y_train, y_test = train_test_split(pad_text, labels, test_size=0.3)
print(x_train.shape)
#x_train=x_train.reshape(32,1000,-1)
#print(x_train.shape)
print(y_train.shape)
#y_train=to_categorical(y_train,2)
#y_test=to_categorical(y_test,2)
#pad_text,labels=torch.tensor(pad_text).float(),torch.LongTensor(labels)

#构建数据迭代器
class Mydataset(torch.utils.data.Dataset):
    def __init__(self, text_list, label_list):
        self.text_list = text_list
        self.label_list = label_list

    def __getitem__(self, index):
        text = torch.LongTensor(self.text_list[index])
        label = self.label_list[index]
        return text, label

    def __len__(self):
        return len(self.text_list)

data_ds = Mydataset(pad_text,labels)
#train_ds = Mydataset(x_train, y_train)
#test_ds = Mydataset(x_test, y_test)

batch_size = 16

data_dl =torch.utils.data.DataLoader(data_ds,batch_size=batch_size,shuffle=False)

# embed_dim = 2 ** (int(np.log2(len(word_list) ** 0.25)) + 2)  # 经验值

embed_dim = 50
hidden_size = 20

class Net(nn.Module):
    def __init__(self, word_list, embed_dim, hidden_size, num_layers=2):
        super().__init__()

        self.em = nn.Embedding(len(word_list) + 1, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_size, num_layers, bidirectional=True)
        self.pool1=nn.MaxPool1d(5)
        self.pool2 = nn.MaxPool1d(3)
        self.linear1 = nn.Linear(2*hidden_size, 128)
        self.linear2 = nn.Linear(128, 2)

    def forward(self, inputs):
        x = self.em(inputs)  # inputs is input, size (seq_len, batch, input_size)               #[16,1200,50]
        x = x.float()
        x, _ = self.gru(x)  # x is outuput, size (seq_len, batch, hidden_size)                  #[16,1200,20]
        x = self.pool1(x)                                                                       #[16,1200,4]
        x =self.pool2(x)                                                                        #[16,1200,1]
        x =x.view(-1,1200)
#         x =x.view(16,-1)
        #x = torch.sum(x, dim=0)                                                                #[1200,20]
        #x = F.relu(self.linear1(x))                                                            #[1200,128]
        #x = self.linear2(x)                                                                    #[1200,2]
        return x

model = Net(word_list, embed_dim, hidden_size)                  #实例化
model = model.to(device)

loss = nn.CrossEntropyLoss()
loss = loss.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

with torch.no_grad():
    outputs=[]
    for x,y in data_dl:
        out = model(x)
        outputs.append(out)

outputs1 = torch.concat(outputs,axis =0)

deep_fea = pd.DataFrame(outputs1)
                                    
deep_fea.to_csv(r'C:\Users\26970\Desktop\特征融合\初始BiGRU所有特征-1200.csv')
deep_fea

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\26970\\Desktop\\特征融合\\基准数据集.csv'

In [206]:
fea_all = feature.join(deep_fea)
fea_all

,GC_con,score,cdsStop,cdsSizes,T,C,GT,GC,ACG,TAT,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,0.378788,1124.0,924,924,0.347403,0.151515,0.063991,0.040130,0.004348,0.023913,...,0.174586,0.593868,0.232614,0.360161,0.308725,0.510515,0.209233,0.378684,0.209874,0.377706
1,0.418803,668.0,468,468,0.254274,0.177350,0.057940,0.032189,0.012931,0.010776,...,0.296095,0.582727,0.338518,0.563349,0.268748,0.613168,0.343790,0.581930,0.334156,0.575203
2,0.420833,820.0,720,720,0.273611,0.179167,0.051532,0.057103,0.005587,0.015363,...,0.349529,0.629848,0.293577,0.564277,0.296527,0.435395,0.332147,0.394462,0.341412,0.390763
3,0.475096,722.0,522,522,0.302682,0.302682,0.044231,0.044231,0.005792,0.017375,...,0.439637,0.611178,0.507852,0.539785,0.434861,0.599211,0.516107,0.564896,0.502040,0.556498
4,0.414545,1850.0,1650,1650,0.285455,0.190909,0.049757,0.037015,0.006075,0.015796,...,0.354306,0.597440,0.315236,0.530688,0.254151,0.444344,0.407017,0.422034,0.423000,0.426615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16115,0.526316,109.0,45,9,0.223684,0.250000,0.054054,0.054054,0.000000,0.000000,...,0.596888,0.603294,0.567875,0.584350,0.589436,0.595875,0.582883,0.593436,0.582407,0.597919
16116,0.487805,223.5,80,75,0.317073,0.207317,0.037500,0.075000,0.000000,0.025641,...,0.553937,0.674886,0.472268,0.587634,0.372444,0.507836,0.398429,0.602019,0.542768,0.557220
16117,0.500000,223.5,80,75,0.304878,0.207317,0.037500,0.075000,0.000000,0.025641,...,0.613887,0.614279,0.595380,0.603822,0.612972,0.609326,0.607958,0.607066,0.608824,0.600273
16118,0.500000,223.5,80,75,0.304878,0.207317,0.037500,0.075000,0.000000,0.025641,...,0.496939,0.636099,0.494469,0.577936,0.383919,0.464381,0.427793,0.601590,0.478494,0.464147


In [207]:
y = pd.DataFrame(labels, columns=['label'])

fea_all.insert(0,'label',y)
fea_all

,label,GC_con,score,cdsStop,cdsSizes,T,C,GT,GC,ACG,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,0,0.378788,1124.0,924,924,0.347403,0.151515,0.063991,0.040130,0.004348,...,0.174586,0.593868,0.232614,0.360161,0.308725,0.510515,0.209233,0.378684,0.209874,0.377706
1,0,0.418803,668.0,468,468,0.254274,0.177350,0.057940,0.032189,0.012931,...,0.296095,0.582727,0.338518,0.563349,0.268748,0.613168,0.343790,0.581930,0.334156,0.575203
2,0,0.420833,820.0,720,720,0.273611,0.179167,0.051532,0.057103,0.005587,...,0.349529,0.629848,0.293577,0.564277,0.296527,0.435395,0.332147,0.394462,0.341412,0.390763
3,0,0.475096,722.0,522,522,0.302682,0.302682,0.044231,0.044231,0.005792,...,0.439637,0.611178,0.507852,0.539785,0.434861,0.599211,0.516107,0.564896,0.502040,0.556498
4,0,0.414545,1850.0,1650,1650,0.285455,0.190909,0.049757,0.037015,0.006075,...,0.354306,0.597440,0.315236,0.530688,0.254151,0.444344,0.407017,0.422034,0.423000,0.426615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16115,1,0.526316,109.0,45,9,0.223684,0.250000,0.054054,0.054054,0.000000,...,0.596888,0.603294,0.567875,0.584350,0.589436,0.595875,0.582883,0.593436,0.582407,0.597919
16116,1,0.487805,223.5,80,75,0.317073,0.207317,0.037500,0.075000,0.000000,...,0.553937,0.674886,0.472268,0.587634,0.372444,0.507836,0.398429,0.602019,0.542768,0.557220
16117,1,0.500000,223.5,80,75,0.304878,0.207317,0.037500,0.075000,0.000000,...,0.613887,0.614279,0.595380,0.603822,0.612972,0.609326,0.607958,0.607066,0.608824,0.600273
16118,1,0.500000,223.5,80,75,0.304878,0.207317,0.037500,0.075000,0.000000,...,0.496939,0.636099,0.494469,0.577936,0.383919,0.464381,0.427793,0.601590,0.478494,0.464147


In [88]:
# 定义朴素贝叶斯模型
from sklearn.metrics import accuracy_score, f1_score, auc, matthews_corrcoef, precision_score, recall_score, confusion_matrix, roc_curve
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_predict

def model_gnb(X, y, X_test, y_test):
    
    X.columns = X.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)
    
    gnb = GaussianNB()

    # 定义评估指标
    scoring = {'accuracy': 'accuracy',
           'f1': 'f1',
           'roc_auc': 'roc_auc',
           'mcc': make_scorer(matthews_corrcoef)}

    # 进行五倍交叉验证
    cv_results = cross_validate(gnb, X, y, scoring=scoring, cv=5, return_train_score=False)

    # 计算并输出评估指标
    # print("Accuracy: %f (+/- %f)" % (cv_results['test_accuracy'].mean(), cv_results['test_accuracy'].std() * 2))
    # print("F1 score: %f (+/- %f)" % (cv_results['test_f1'].mean(), cv_results['test_f1'].std() * 2))
    # print("ROC AUC score: %f (+/- %f)" % (cv_results['test_roc_auc'].mean(), cv_results['test_roc_auc'].std() * 2))
    # print("MCC: %f (+/- %f)" % (cv_results['test_mcc'].mean(), cv_results['test_mcc'].std() * 2))

    print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
    print("F1 score: %f" % (cv_results['test_f1'].mean()))
    print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
    print("MCC: %f" % (cv_results['test_mcc'].mean()))

    # 计算混淆矩阵
    y_pred = cross_val_predict(gnb, X, y, cv=5)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    npv = tn / (tn + fn)
    ppv = tp / (tp + fp)
    se = tp / (tp + fn)
    spc = tn / (tn + fp)

    # 输出混淆矩阵及相关指标
    print(f"NPV: {npv:f}")
    print(f"PPV: {ppv:f}")
    print(f"Sensitivity (Recall): {se:f}")
    print(f"Specificity: {spc:f}")

    # 在独立测试集上评估模型
    gnb.fit(X, y)
    y_pred_test = gnb.predict(X_test)
    auc_score_test = roc_auc_score(y_test, y_pred_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    mcc_test = matthews_corrcoef(y_test, y_pred_test)

    # 输出独立测试集上的评估指标
    print("Test Set - Accuracy: %f" % (accuracy_test))
    print("Test Set - F1 score: %f" % (f1_score_test))
    print("Test Set - ROC AUC score: %f" % (auc_score_test))
    print("Test Set - MCC: %f" % (mcc_test))
    
    # 计算混淆矩阵和评估指标（独立测试集）
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    npv_test = tn_test / (tn_test + fn_test)
    ppv_test = tp_test / (tp_test + fp_test)
    se_test = tp_test / (tp_test + fn_test)
    spc_test = tn_test / (tn_test + fp_test)

    # 输出混淆矩阵及相关指标（独立测试集）
    print(f"Test Set - NPV: {npv_test:f}")
    print(f"Test Set - PPV: {ppv_test:f}")
    print(f"Test Set - Sensitivity (Recall): {se_test:f}")
    print(f"Test Set - Specificity: {spc_test:f}")
    
model_gnb(ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0])

D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Accuracy: 0.719000
F1 score: 0.702569
ROC AUC score: 0.780305
MCC: 0.445082


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

NPV: 0.698594
PPV: 0.744079
Sensitivity (Recall): 0.667625
Specificity: 0.770375


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Test Set - Accuracy: 0.533235
Test Set - F1 score: 0.677636
Test Set - ROC AUC score: 0.541584
Test Set - MCC: 0.205090
Test Set - NPV: 0.993151
Test Set - PPV: 0.512600
Test Set - Sensitivity (Recall): 0.999401
Test Set - Specificity: 0.083767


In [89]:
# 定义SVM模型

def model_SVM(X, y, X_test, y_test):
    from sklearn.svm import SVC
    from sklearn.model_selection import cross_validate
    from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, precision_score, recall_score
    from sklearn.model_selection import cross_val_predict
    
    X.columns = X.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)
    
    # 创建SVM分类器
    svm_clf = SVC()

    # 定义评估指标
    scoring = {'accuracy': 'accuracy',
           'f1': 'f1',
           'roc_auc': 'roc_auc',
           'mcc': make_scorer(matthews_corrcoef)}

    # 进行5倍交叉验证并计算评估指标
    cv_results = cross_validate(svm_clf, X, y, cv=5, scoring=scoring)

    # 输出评估指标的平均值和标准差
    print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
    print("F1 score: %f" % (cv_results['test_f1'].mean()))
    print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
    print("MCC: %f" % (cv_results['test_mcc'].mean()))

    # 计算混淆矩阵
    y_pred = cross_val_predict(svm_clf, X, y, cv=5)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    npv = tn / (tn + fn)
    ppv = tp / (tp + fp)
    se = tp / (tp + fn)
    spc = tn / (tn + fp)

    # 输出混淆矩阵及相关指标
    print(f"NPV: {npv:f}")
    print(f"PPV: {ppv:f}")
    print(f"Sensitivity (Recall): {se:f}")
    print(f"Specificity: {spc:f}")

    # 在独立测试集上评估模型
    svm_clf.fit(X, y)
    y_pred_test = svm_clf.predict(X_test)
    auc_score_test = roc_auc_score(y_test, y_pred_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    mcc_test = matthews_corrcoef(y_test, y_pred_test)

    # 输出独立测试集上的评估指标
    print("Test Set - Accuracy: %f" % (accuracy_test))
    print("Test Set - F1 score: %f" % (f1_score_test))
    print("Test Set - ROC AUC score: %f" % (auc_score_test))
    print("Test Set - MCC: %f" % (mcc_test))
    
    # 计算混淆矩阵和评估指标（独立测试集）
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    npv_test = tn_test / (tn_test + fn_test)
    ppv_test = tp_test / (tp_test + fp_test)
    se_test = tp_test / (tp_test + fn_test)
    spc_test = tn_test / (tn_test + fp_test)

    # 输出混淆矩阵及相关指标（独立测试集）
    print(f"Test Set - NPV: {npv_test:f}")
    print(f"Test Set - PPV: {ppv_test:f}")
    print(f"Test Set - Sensitivity (Recall): {se_test:f}")
    print(f"Test Set - Specificity: {spc_test:f}")
    
model_SVM(ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0])

D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Accuracy: 0.928125
F1 score: 0.933050
ROC AUC score: 0.967702
MCC: 0.865401


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

NPV: 1.000000
PPV: 0.874317
Sensitivity (Recall): 1.000000
Specificity: 0.856250


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Test Set - Accuracy: 0.914471
Test Set - F1 score: 0.919757
Test Set - ROC AUC score: 0.915977
Test Set - MCC: 0.841334
Test Set - NPV: 0.998339
Test Set - PPV: 0.852481
Test Set - Sensitivity (Recall): 0.998562
Test Set - Specificity: 0.833391


In [208]:
### RFC训练模型

def model_RFC(X, y, X_test, y_test):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import cross_validate
    from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, precision_score, recall_score
    from sklearn.model_selection import cross_val_predict

    X.columns = X.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)
    
    # 创建随机森林分类器
    rf_clf = RandomForestClassifier()

    # 定义评估指标
    scoring = {'accuracy': 'accuracy',
           'f1': 'f1',
           'roc_auc': 'roc_auc',
           'mcc': make_scorer(matthews_corrcoef)}

    # 进行5倍交叉验证并计算评估指标
    cv_results = cross_validate(rf_clf, X, y, cv=5, scoring=scoring)

    # 输出评估指标的平均值和标准差
    print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
    print("F1 score: %f" % (cv_results['test_f1'].mean()))
    print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
    print("MCC: %f" % (cv_results['test_mcc'].mean()))

    # 计算混淆矩阵
    y_pred = cross_val_predict(rf_clf, X, y, cv=5)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    npv = tn / (tn + fn)
    ppv = tp / (tp + fp)
    se = tp / (tp + fn)
    spc = tn / (tn + fp)

    # 输出混淆矩阵及相关指标
    print(f"NPV: {npv:f}")
    print(f"PPV: {ppv:f}")
    print(f"Sensitivity (Recall): {se:f}")
    print(f"Specificity: {spc:f}")
    
    # 在独立测试集上评估模型
    rf_clf.fit(X, y)
    y_pred_test = rf_clf.predict(X_test)
    auc_score_test = roc_auc_score(y_test, y_pred_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    mcc_test = matthews_corrcoef(y_test, y_pred_test)

    # 输出独立测试集上的评估指标
    print("Test Set - Accuracy: %f" % (accuracy_test))
    print("Test Set - F1 score: %f" % (f1_score_test))
    print("Test Set - ROC AUC score: %f" % (auc_score_test))
    print("Test Set - MCC: %f" % (mcc_test))
    
    # 计算混淆矩阵和评估指标（独立测试集）
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    npv_test = tn_test / (tn_test + fn_test)
    ppv_test = tp_test / (tp_test + fp_test)
    se_test = tp_test / (tp_test + fn_test)
    spc_test = tn_test / (tn_test + fp_test)

    # 输出混淆矩阵及相关指标（独立测试集）
    print(f"Test Set - NPV: {npv_test:f}")
    print(f"Test Set - PPV: {ppv_test:f}")
    print(f"Test Set - Sensitivity (Recall): {se_test:f}")
    print(f"Test Set - Specificity: {spc_test:f}")

model_RFC(ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0])

D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarni

Accuracy: 0.955438
F1 score: 0.956064
ROC AUC score: 0.983100
MCC: 0.913233


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionW

NPV: 0.967531
PPV: 0.943835
Sensitivity (Recall): 0.968375
Specificity: 0.942375


C:\Users\26970\AppData\Local\Temp\ipykernel_5144\1020548416.py:46: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf.fit(X, y)


Test Set - Accuracy: 0.877543
Test Set - F1 score: 0.887917
Test Set - ROC AUC score: 0.880761
Test Set - MCC: 0.779052
Test Set - NPV: 0.997006
Test Set - PPV: 0.799980
Test Set - Sensitivity (Recall): 0.997576
Test Set - Specificity: 0.763946
